In [1]:
import json
from src.surquest.utils.appstoreconnect.credentials import Credentials
from src.surquest.utils.appstoreconnect.analyticsreports.client import Client
from src.surquest.utils.appstoreconnect.analyticsreports.handler import Handler
from src.surquest.utils.appstoreconnect.analyticsreports.enums.category import Category
from src.surquest.utils.appstoreconnect.analyticsreports.enums.granularity import Granularity
from src.surquest.utils.appstoreconnect.analyticsreports.enums.report_name import ReportName
from pathlib import Path

In [ ]:
key_path = Path.cwd() / "credentials" / "key.p8"
CATEGORY = Category.FRAMEWORK_USAGE
REPORT_NAME = ReportName.APP_SESSIONS_STANDARD
GRANULARITY = Granularity.DAILY

APP_ID = "950949627"  # LRT
# APP_ID = "6451202232" # MW
# APP_ID = "1207764294" # LGO


KEY_ID = '5WDUV3USAU' # 10-character key ID from App Store Connect
ISSUER_ID = '69a6de80-fd44-47e3-e053-5b8c7c11a4d1'  # 36-character issuer ID
PRIVATE_KEY = key_path.read_text()
CSV_PATH = Path.cwd() / "data" / REPORT_NAME.name / f"data.{APP_ID}.csv"
JSON_PATH = Path.cwd() / "data" / REPORT_NAME.name / f"data.{APP_ID}.jsonl"

credentials = Credentials(
    issuer_id=ISSUER_ID, # 36-character issuer ID
    key_id=KEY_ID, # 10-character key ID from App Store Connect
    private_key=PRIVATE_KEY
)

token = credentials.generate_token()
client = Client(credentials=credentials)
# client.create_reports_request(
#     app_id=APP_ID,
#     access_type="ONE_TIME_SNAPSHOT"
# )
# data = client._get_resource(
#     resource_path=f"apps/{APP_ID}/customerReviews",
#     params={
#         "limit": 200,
#         "sort": "-createdDate",
#         "include": "response",
#         "fields[customerReviewResponses]": "responseBody,lastModifiedDate,state,review",
#         "fields[customerReviews]": "rating,title,body,reviewerNickname,createdDate,territory,response"
#     }
# )

# data = client.fetch_customer_reviews(
#     app_id=APP_ID, 
#     last_known_customer_review_id="00000038-ae56-fb03-073b-3efc00000000",
#     max_iterations=10
#     )

# print(json.dumps(data, indent=4))

In [ ]:
client.read_report_requests(
    app_id=APP_ID,
    access_type="ONE_TIME_SNAPSHOT"
)

In [ ]:

data = client.get_data(
    app_id = APP_ID,
    report_name = REPORT_NAME,
    # access_type = "ONE_TIME_SNAPSHOT"
)

Handler.list_of_dicts_to_csv(data, CSV_PATH)
Handler.list_of_dicts_to_jsonl(data, JSON_PATH)

In [ ]:
reports = Handler.extract_attribute_values(client.list_reports(
    app_id=APP_ID
))

for i in reports:
    print(f"{i['name'].upper().replace(' ', '_')} = (\"{i['name']}\",  Category.{i['category']})")

In [ ]:
from enum import Enum

class App(str, Enum):

    LRT = "950949627"
    LGO = "1207764294"
    MV = "6451202232"

for app in [App.LRT, App.LGO, App.MV]:

    report_requests = client.read_report_requests(
        app_id=app.value
    )

    print(f"{app.name}: report requests {len(report_requests)} - {report_requests}")


    if len(report_requests) == 0:

        client.create_report_request(app_id = app.value)

In [ ]:
app = "6451202232"
report_requests = client.read_report_requests(
    app_id=app
)

report_requests
print(report_requests)
report_requests_id = report_requests[0].get("id")

specific_request = client.read_report_for_specific_request(
    report_requests_id,
    params={
        "filter[category]": REPORT_NAME.category.value,
        "filter[name]":REPORT_NAME.value
    }
)
print(Handler.extract_attribute_values(specific_request))
specific_request_id = specific_request[0].get("id")
specific_request_ids = Handler.extract_ids(specific_request)

for i in specific_request_ids:

    instances = client.read_list_of_instances_of_report(
        specific_request_id,
        params={"filter[granularity]": "DAILY"}
    )

    for inst in instances:
        print(inst)
        inst_id = inst.get('id')

        response_url = client._fetch_segment_urls([inst_id])
        print(response_url)

In [ ]:
#1A Read Report Requests
report_requests = client.read_report_requests(app_id=APP_ID)
report_request_id = report_requests.get("data")[0].get("id")
RenderJSON(report_requests)


In [ ]:
#2A Read Report
report = client.read_report_for_specific_request(
    request_id=report_request_id,
    params={
        "filter[category]": CATEGORY.value,
        "filter[name]": REPORT_NAME
    }
)

report_id = report.get("data")[0].get("id")
print(report_id)
print(report)
RenderJSON(report)

In [ ]:
#3A Read Report Instances
instances = client.read_list_of_instances_of_report(
    report_id=report_id,
    params={
        "filter[granularity]": GRANULARITY.value,
        "filter[processingDate]": DATE
    }
)
instance_id = instances.get("data")[0].get("id")
RenderJSON(instances)

In [ ]:
#4 Get segments
segments = client.read_segments_for_report(
    instance_id=instance_id
)
url  = segments.get("data")[0].get("attributes").get("url")
RenderJSON(segments)


In [ ]:
# Download the report data
data = client.download_report_to_dicts(
    report_url=url
)
data

In [ ]:
client.get_data(
    app_id = APP_ID,
    report_name = REPORT_NAME,
    category = CATEGORY,
    granularity = GRANULARITY,
    dates = {'2025-07-25', '2025-07-26'}
)
